<a href="https://colab.research.google.com/github/AsmaMora/Customer_Analytics/blob/main/MADT8101_Voice_of_customer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --pre pythainlp
!pip install pyLDAvis

In [ ]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
df=pd.read_csv('C:\\Asma\\MADT\\Semester 3\\MADT8101\\Class 14\\Got_to_grill_review.csv')

In [ ]:
df.tail()

,ReviewID,Review
5,6,ร้านน่านั่ง อาหารอร่อย ไม่ต้องรอนาน
6,7,อาหารญี่ปุ่น ปิ้งย่างฮาลาล ที่สุดใน กรุงเทพ
7,8,เนื้อนุ่ม พนักงานบริการช้านิดนึง แต่ก็โอเคค่ะ
8,9,บุฟเฟ่ต์ฮาลาล เนื้อย่างพรีเมียม สายเนื้อต้องไม...
9,10,เนื้อคือดีมาก


In [ ]:
#Tokenize Words
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = ['',' ','  ','\n','ค่ะ','คะ','ร้าน','นะคะ','ๆ','('')','(',')','(' ,')','(' , ')','(', ')','โอเค','"','"', 'กก' , 'Got' , 'To' , 'Grill']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
    merged = ''
    words = pythainlp.word_tokenize(str(sentence), engine='newmm')
    for word in words:
        if word not in screening_words:
            merged = merged + ',' + word
    return merged[1:]

In [ ]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [ ]:
df.tail()

,ReviewID,Review,Review_tokenized,topics,score
5,6,ร้านน่านั่ง อาหารอร่อย ไม่ต้องรอนาน,"นั่ง,อาหาร,อร่อย,ไม่ต้อง,รอ",2,0.012955
6,7,อาหารญี่ปุ่น ปิ้งย่างฮาลาล ที่สุดใน กรุงเทพ,"อาหาร,ญี่ปุ่น,ปิ้ง,ย่าง,ฮาลาล,กรุงเทพ",1,0.970423
7,8,เนื้อนุ่ม พนักงานบริการช้านิดนึง แต่ก็โอเคค่ะ,"เนื้อ,นุ่ม,พนักงานบริการ,นิดนึง",0,0.011247
8,9,บุฟเฟ่ต์ฮาลาล เนื้อย่างพรีเมียม สายเนื้อต้องไม...,"บุฟเฟ่ต์,ฮาลาล,เนื้อ,ย่าง,พรีเมียม,สาย,เนื้อ,พ...",3,0.985159
9,10,เนื้อคือดีมาก,"เนื้อ,ดีมาก",0,0.915878


In [ ]:
#Create Dictionary
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [ ]:
print(dictionary.token2id.keys())

dict_keys(['Garden', 'Loft', 'บุฟเฟต์', 'พรีเมี่ยม', 'ราคา', 'อร่อย', 'ฮาลาล', 'แนว', 'ซี', 'ซูชิ', 'ถ้วม', 'บุ', 'ฟู้ด', 'ฟเฟ่ห์', 'ย่าง', 'เนื้อ', 'ไอศครีม', 'กะปิ', 'กิว', 'คุณภาพดี', 'บุฟเฟ่ต์', 'ม่อน', 'ย่าน', 'รามคำแหง', 'ล', 'วา', 'สาย', 'แซ', '399', 'คุ้ม', 'โปร', 'ซา', 'ซิ', 'ดีมาก', 'สด', 'เมนู', 'เลือก', 'นั่ง', 'รอ', 'อาหาร', 'ไม่ต้อง', 'กรุงเทพ', 'ญี่ปุ่น', 'ปิ้ง', 'นิดนึง', 'นุ่ม', 'พนักงานบริการ', 'to', 'พรีเมียม', 'พลาด'])


In [ ]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [ ]:
#Topic Modeling
num_topics = 4
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every =1 # Don't evaluate model perplexity, takes too much time

#make a index to word dictionary
temp = dictionary[0] #This is only to load the dictionary
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                                    alpha='auto', eta='auto', \
                                    iterations=iterations, num_topics=num_topics, \
                                    passes=passes, eval_every=eval_every)

CPU times: total: 46.9 ms
Wall time: 54.7 ms


In [ ]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.107949 -0.002471       1        1  53.421163
0     -0.048117  0.066590       2        1  21.751098
2     -0.071513 -0.050722       3        1  21.750576
1      0.011681 -0.013397       4        1   3.077164, topic_info=             Term      Freq     Total Category  logprob  loglift
15          เนื้อ  4.000000  4.000000  Default  30.0000  30.0000
33          ดีมาก  1.000000  1.000000  Default  29.0000  29.0000
12           ฟู้ด  1.000000  1.000000  Default  28.0000  28.0000
39          อาหาร  1.000000  1.000000  Default  27.0000  27.0000
8              ซี  1.000000  1.000000  Default  26.0000  26.0000
..            ...       ...       ...      ...      ...      ...
46  พนักงานบริการ  0.036608  1.240288   Topic4  -4.0604  -0.0417
29           คุ้ม  0.036608  1.240288   Topic4  -4.0604  -0.0417
12           ฟู้ด  0.036608  1.587161   Topic4  -4.0604  -0.2883
8              ซี  0.036608  1.587161   Topic4  -4.0604  -0.2883
0          Garden  0.036608  1.240289   Topic4  -4.0604  -0.0417

[184 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
28        1  0.806264            399
0         1  0.806264         Garden
1         1  0.806263           Loft
47        1  0.806263             to
41        3  0.960248        กรุงเทพ
17        1  0.806277           กะปิ
18        1  0.806282            กิว
19        1  0.806273       คุณภาพดี
29        1  0.806264           คุ้ม
31        3  0.960248             ซา
32        3  0.960248             ซิ
8         2  0.630056             ซี
8         3  0.630056             ซี
9         2  0.960235           ซูชิ
42        3  0.960248        ญี่ปุ่น
33        3  0.841866          ดีมาก
10        2  0.960235           ถ้วม
37        2  0.960235           นั่ง
44        1  0.806264         นิดนึง
45        1  0.806264           นุ่ม
11        2  0.960235             บุ
2         1  0.806263        บุฟเฟต์
20        1  1.007603       บุฟเฟ่ต์
43        3  0.960248           ปิ้ง
46        1  0.806264  พนักงานบริการ
48        1  0.806263       พรีเมียม
3         1  0.806263      พรีเมี่ยม
49        1  0.806263           พลาด
12        2  0.630056           ฟู้ด
12        3  0.630056           ฟู้ด
13        2  0.960235         ฟเฟ่ห์
21        1  0.806278           ม่อน
14        1  0.428849           ย่าง
14        2  0.428849           ย่าง
14        3  0.428849           ย่าง
22        1  0.806276           ย่าน
38        2  0.960235             รอ
4         1  0.806264           ราคา
23        1  0.806279       รามคำแหง
24        1  0.806285              ล
25        1  0.806275             วา
34        3  0.960248             สด
26        1  1.007598            สาย
5         1  0.790289          อร่อย
5         2  0.395145          อร่อย
39        2  0.630056          อาหาร
39        3  0.630056          อาหาร
6         1  0.650090          ฮาลาล
6         2  0.325045          ฮาลาล
6         3  0.325045          ฮาลาล
15        1  0.636591          เนื้อ
15        2  0.212197          เนื้อ
15        3  0.212197          เนื้อ
35        3  0.960248           เมนู
36        3  0.960248          เลือก
27        1  0.806276             แซ
7         1  0.806264            แนว
30        1  0.806265            โปร
40        2  0.960235        ไม่ต้อง
16        2  0.960235        ไอศครีม, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2])

In [ ]:
model.show_topic(1)

[('ดีมาก', 0.08620604),
 ('เนื้อ', 0.08616695),
 ('ล', 0.01724548),
 ('กิว', 0.017244827),
 ('บุฟเฟ่ต์', 0.017244752),
 ('รามคำแหง', 0.01724429),
 ('ม่อน', 0.01724419),
 ('กะปิ', 0.017244035),
 ('ย่าน', 0.017243834),
 ('แซ', 0.017243799)]

In [ ]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [ ]:
df.tail()

,ReviewID,Review,Review_tokenized,topics,score
5,6,ร้านน่านั่ง อาหารอร่อย ไม่ต้องรอนาน,"นั่ง,อาหาร,อร่อย,ไม่ต้อง,รอ",0,0.959096
6,7,อาหารญี่ปุ่น ปิ้งย่างฮาลาล ที่สุดใน กรุงเทพ,"อาหาร,ญี่ปุ่น,ปิ้ง,ย่าง,ฮาลาล,กรุงเทพ",2,0.965635
7,8,เนื้อนุ่ม พนักงานบริการช้านิดนึง แต่ก็โอเคค่ะ,"เนื้อ,นุ่ม,พนักงานบริการ,นิดนึง",0,0.013058
8,9,บุฟเฟ่ต์ฮาลาล เนื้อย่างพรีเมียม สายเนื้อต้องไม...,"บุฟเฟ่ต์,ฮาลาล,เนื้อ,ย่าง,พรีเมียม,สาย,เนื้อ,พ...",3,0.983842
9,10,เนื้อคือดีมาก,"เนื้อ,ดีมาก",0,0.024557
